
# Imports and class definition for the overlay


Multiple excutions without a board restart can put the "HDMI" Unit in an undefined state as the state of the internal units is not reset properly by redownloading the overlay. This will result in the misallignment of the output hdmi stream or crashes of the python kernel. This can be fixed through restarting the board. 

In [ ]:
import pynq
from pynq.mmio import MMIO
from pynq.lib.video import *

import numpy as np

import cv2

import time

from stewart_controller import Stewart_Platform
from pipelineOverlay import PipelineOverlay
from const import *
from servo import *

import math

import collections

from IPython.display import clear_output

from simple_pid import PID

## Board Setup
Load the bitfile and calibrate the center of the frame post filters.

In [ ]:
# Load Base Overlay
base = PipelineOverlay("/home/xilinx/jupyter_notebooks/bit_files/top_level.bit")

base.determine_inital_center(False)

# Setup Video
## Setup of the AXIS Switch
 
 The AXIS Switch configuration connects outputs(master) to inputs(slave). Every output is 
 associated with a specific memory mapped register and the value in this register determines 
 which input is connted to this output.</br>

 Reference address `0x40000000` </br>
 Starting offset for output registers: `0x40` </br>
 Every 4 byte is a new register </br>
   => offset `0x40` => `M0` with value = 1 => `M0` connected to `S1` </br>
   => offset `0x44` => `M1` with value = 2 => `M1` connected to `S2` </br>
 
 The register changes also need to be committed before they take effect.
 For this the register at offset `0x0` needs to be set to `0x2`.
 This register is self resetting.

### Input
- S0 Video Input
- S1 Smoothing Ouput
- S2 Edge Output
- S3 Circle Detection
- S4 Smoothing
- S5 Smoothing
- S6 Smoothing
- S7 Test Filter
- S8 Crop

### Outputs
- M0 Video Output
- M1 Smoothing Input
- M2 Edge Input
- M3 Circle Detection
- M4 Smoothing
- M5 Smoothing
- M6 Smoothing
- M7 Test Filter
- M8 Crop

In [ ]:
# Axis Switch need to be setup before video in is enabled
print("Reset  Axis Switch")
base.axis_switch_reset()

print("Set Axis Switch")        
base.axis_switch_setup_connect(INPUT         , OUTPUT)

print("Commit Axis Switch")
base.axis_switch_commit()

print("Start Video In")
base.start_video_in()

### Set Crop limits
The limits are set based on a range around the detected center

In [4]:
x_radius = 330
y_radius = 330
base.set_crop_limits( 
    base.center_x - x_radius, 
    base.center_x + x_radius, 
    base.center_y - y_radius, 
    base.center_y + y_radius)

# LOOP
This is the main processing loop, which is responsible for controlling the filter pipeline. Adding and removing filters is done via the buttons on the board. The script gets the current ball position and calculates the needed angles for every servo and sets these values for the pwm module.

The values used for the Stewart Platform need to be adjusted before use.

In [ ]:
# The parameters here need to be adjusted to work with the platform
#   Radius of the Base Servo Position
#   Radius of the anchor points on the Platform
#   Servo arm length
#   Platform leg length
#   Half the angle between servos in radian
#   Half the angle between anchor in radian
platform = Stewart_Platform(19.967, 10.500, 2.5, 33, 0.2269, 0.2269, -(90/180)*np.pi)

filter_array = [False, False, False, False]

button_prev = [0,0,0,0]
switch_prev = [0,0]

changed_filter = False

last_pos = np.array(
    [
        base.circle_detect.read(AXI_CIRCLE_DETECT_X_REG), 
        base.circle_detect.read(AXI_CIRCLE_DETECT_Y_REG)
    ]
)
cur_pos = last_pos

movement_vec = np.array([0,0])

pid_1 = PID(0.0003, 0, 0.00017, setpoint=0, sample_time=None)
pid_2 = PID(0.0003, 0, 0.00017, setpoint=0, sample_time=None)

low_pass_factor = 0.4

pos_low_pass_x = base.center_x
pos_low_pass_y = base.center_y


print("Starting")
while True:  
    time.sleep(1.0/30)
    
    ###########################
    # Button Logic to switch filters
    ###########################
    # Detect a rising edge
    if switch_prev[0] != base.switches[0].read():
        switch_prev[0] = base.switches[0].read()
        changed_filter = True
        
    if switch_prev[1] != base.switches[1].read():
        switch_prev[1] = base.switches[1].read()
        changed_filter = True
    
    # If a rising edge was detected update the filter array
    for i in range(0,4):
        button_val = base.buttons[i].read()
        if button_val == 1 and button_prev[i] == 0:
            changed_filter = True
            filter_array[i] = not filter_array[i]
        button_prev[i] = button_val
    
    # Set the leds for the active filters
    for i in range(0,4):
        if filter_array[i]:
            base.leds[i].on()
        else:
            base.leds[i].off()
    
    # If a change was done to the filters array update the AXIS Switch
    if changed_filter:
        changed_filter = False
        filters = []
        filters.append(OUTPUT)
        
        if filter_array[SMOOTHING_1]:
            filters.append(SMOOTHING_1)
            if base.switches[0].read() == 1 or base.switches[1].read():
                filters.append(SMOOTHING_2)
            if base.switches[1].read() == 1:
                filters.append(SMOOTHING_3)
            if base.switches[1].read() == 1 and base.switches[0].read() == 1:
                filters.append(SMOOTHING_4)
        
        if filter_array[EDGE]:
            filters.append(EDGE)
        
        if filter_array[CROP_BUTTON]:
            filters.append(CROP_PIPELINE)
        
        if filter_array[CIRCLE_DETECT]:
            filters.append(CIRCLE_DETECT)
        
        
        filters.append(INPUT)
        
        # Reset switch
        base.axis_switch_reset()
        
        # Connecte the filter to each other
        prev_element = 255
        for e in filters:
            if prev_element == 255:
                prev_element = e
            else:
                base.axis_switch_setup_connect(prev_element, e)
                prev_element = e

        base.axis_switch_commit()
           
        
    ###########################
    # Position Processing
    ###########################
    raw_ball_pos_x = base.circle_detect.read(AXI_CIRCLE_DETECT_X_REG)
    raw_ball_pos_y = base.circle_detect.read(AXI_CIRCLE_DETECT_Y_REG)
    
    # Low pass filter the position
    if not (raw_ball_pos_x == 0 or raw_ball_pos_y == 0):
        pos_low_pass_x = pos_low_pass_x * (1 - low_pass_factor) + raw_ball_pos_x * low_pass_factor 
        pos_low_pass_y = pos_low_pass_y * (1 - low_pass_factor) + raw_ball_pos_y * low_pass_factor 
    
    ball_pos_x = pos_low_pass_x    
    ball_pos_y = pos_low_pass_y
    
    # IF one of the positions is do not calculate, because not position was detected
    if True: #not (ball_pos_x == 0 or ball_pos_y == 0):
        
        # If the ball moves in the x direction, we need to rotate the y axis
        # and if the ball moves into the y direction we need to rotate around the x axis
        alpha_y = pid_1(ball_pos_x - base.center_x) # zero x position is to the left
        alpha_x = pid_2(base.center_y - ball_pos_y) # zero y is to the top

        servo_angles = reorder_servo_signals(
            platform.calculate( 
                np.array([0,0,0]), 
                np.array([-alpha_x, alpha_y, 0]) 
            )
        )
        
        # The inverse kinematics can output NaN values if the angles become to large
        # If this happend just skip this position 
        if True not in [math.isnan(x) for x in servo_angles]:
            clear_output(wait=True)
            for i in range(6):
                value = radian_to_pwm(servo_angles[i], i)
                mapped_value =  map_servo_pwm_signal(i, value)
                base.pwm_controller_set(i, mapped_value)
            print(f"{[ball_pos_x, ball_pos_y]} {[ball_pos_x - base.center_x, base.center_y - ball_pos_y]} {alpha_x} {alpha_y}")
            base.pwm_controller_commit()
        else:
            print("Servo angles have NaN!")

  
    